In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk import word_tokenize

/home/visal/anaconda3/envs/keras/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('Geo_Cambo_v2.csv', dtype='str')

In [3]:
df['village_name'] = df['village_name'].str.replace('\"', "", regex=True)
df['village_name'] = df['village_name'].str.replace('(', "", regex=True)
df['village_name'] = df['village_name'].str.replace(')', "", regex=True)
df['village_name'] = df['village_name'].str.replace('-', "", regex=True)

In [4]:
df.columns

Index(['com_id', 'com_com', 'dis_id', 'dis_name', 'pro_id', 'pro_name',
       'village_id', 'village_name'],
      dtype='object')

In [5]:
vocabs = list(pd.concat([df['pro_name'].drop_duplicates(), df['dis_name'].drop_duplicates(),
                df['com_com'].drop_duplicates(), df['village_name'].drop_duplicates()]).drop_duplicates())

In [6]:
vocabs_df = pd.concat([df['pro_name'].drop_duplicates(), df['dis_name'].drop_duplicates(),
                df['com_com'].drop_duplicates(), df['village_name'].drop_duplicates()]).to_frame(name='vocab')

In [7]:
for col in ['com_id', 'dis_id', 'pro_id', 'village_id']:
    df[col] = '0'+df[col] 

In [8]:
pro = df[['pro_id', 'pro_name']].drop_duplicates()

In [9]:
pro_dict = dict(zip(df.pro_id, df.pro_name))
dist_dict = dict(zip(df.dis_id, df.dis_name))
com_dict = dict(zip(df.com_id, df.com_com))
vill_dict = dict(zip(df.village_id, df.village_name))

In [10]:
dictionary = {
    'Province' : list(df.pro_name.drop_duplicates()),
    'District' : list(df.dis_name.drop_duplicates()),
    'Commune' : list(df.com_com.drop_duplicates()),
    'Village' : list(df.village_name.drop_duplicates())
}

In [11]:
# iterating through list1 to extract 
# it's closest match from list2
add = ["Kraties"]
result = []
for i in add:
    f = []
    pro = process.extract(i, pro_dict, limit=1, scorer=fuzz.ratio)[0][0]
    dist = process.extract(i, dist_dict, limit=1, scorer=fuzz.ratio)[0][0]
    com = process.extract(i, com_dict, limit=1, scorer=fuzz.ratio)[0][0]
    vill = process.extract(i, vill_dict, limit=1, scorer=fuzz.ratio)[0][0]
    f.append(pro)
    f.append(dist)
    f.append(com)
    f.append(vill)
    print(process.extract(i, f, limit=1))

[('Kratie', 92)]


In [12]:
confidenceLevels = pd.DataFrame({
    'wordLength' : [1,2,3,4,5,6,7,8,9,10], 
    'minRatio' : [100,100,100,100,80,80,80,80,75,75]
})

def getFuzzyRatio(token = None, confidence_levels = True, default_level = 85):
    """
    This function is meant to retrieve the matching minimum similarity ratio for a particular string length. 
    As string length decreases, you should work with higher ratios to ensure you are not matching words that shouldn't match.
    """
    
    # Check for appropriate formats
    assert isinstance(token, str), "Tokens can be str() type only"
    
    # We check if confidence levels are set
    if confidence_levels:
        for i, row in confidenceLevels.iterrows():
            if len(token) > confidenceLevels['wordLength'].max():
                min_ratio = confidenceLevels['minRatio'].min()
            else:
                min_ratio = confidenceLevels.loc[confidenceLevels['wordLength'] == len(token)]['minRatio'].values[0]
                
    # Fallback if confidence levels are not set
    else:
        min_ratio = default_level
        
    return int(min_ratio)

def getFuzzySimilarity(token = None, dictionary = None, min_ratio = None):
    """
    This function uses the FuzzyWuzzy library to find the highest matching similary score of a token in a list of values.
    We then compare the similary score of the fuzzy match with our minimum threshold and return a match if the match > treshold.
    """
    
    # Check for appropriate formats
    assert isinstance(token, str), "Tokens can be str() type only"
    assert isinstance(dictionary, dict), "Dictionary format should be provided in the dictionary parameter."
    assert isinstance(min_ratio, int), "Integer format should be provided in the minimum-ratio parameter."
    
    for key, values in dictionary.items():
        # Using the process option of FuzzyWuzzy, we can search through the entire dictionary for the best match
        match = process.extractOne(token, values, scorer = fuzz.ratio)
        # Match is a tuple with the match value and the similary score.
        if min_ratio <= match[1]:
            return (match + (key, ))  

tokens = ["Chhroy Basak"]
for token in tokens:
    fuzzy_ratio = getFuzzyRatio(token = token, confidence_levels = True)
    similarity_score = getFuzzySimilarity(token = token, dictionary = dictionary, min_ratio = fuzzy_ratio)
    if not similarity_score == None:
        print("'" + token + "'" + " matches with " + "'"  + str(similarity_score[0]) + "'" + " in [" + similarity_score[2] + "]" + " with a score of: " + str(similarity_score[1]))              

'Chhroy Basak' matches with 'Chrey Bak' in [Commune] with a score of: 76


In [13]:
address = 'Phum Trea Village, Stueng Mean Chey Commune, Meanchey District, Phnom Penh Province'
address1 = 'Trea'
for i in address1.split(','):
    f = []
    pro = process.extract(i, pro_dict, limit=1, scorer=fuzz.ratio)[0][0]
    dist = process.extract(i, dist_dict, limit=1, scorer=fuzz.ratio)[0][0]
    com = process.extract(i, com_dict, limit=1, scorer=fuzz.ratio)[0][0]
    vill = process.extract(i, vill_dict, limit=1, scorer=fuzz.ratio)[0][0]
    f.append(pro)
    f.append(dist)
    f.append(com)
    f.append(vill)
    print(pro, ',', dist, ',' , com,',' , vill)
    print(process.extract(i, f, limit=1))

Siem Reap , Treang , Trea , Trea
[('Trea', 100)]


In [15]:
address = 'Trea, Stueng Mean Chey, Meanchey, Phnom Penh'
address1 = 'Trea'
for i in address.split(','):
    f = []
    pro = process.extract(i, pro_dict, limit=1, scorer=fuzz.ratio)[0][0]
    dist = process.extract(i, dist_dict, limit=1, scorer=fuzz.ratio)[0][0]
    com = process.extract(i, com_dict, limit=1, scorer=fuzz.ratio)[0][0]
    vill = process.extract(i, vill_dict, limit=1, scorer=fuzz.ratio)[0][0]
    f.append(pro)
    f.append(dist)
    f.append(com)
    f.append(vill)
    print(process.extract(i, f, limit=1))

[('Trea', 100)]
[('Stueng Mean chey 1', 95)]
[('Mean Chey', 94)]
[('Phnom Penh', 100)]
